<a href="https://colab.research.google.com/github/CamelGoong/Pandas_Practicing/blob/main/PART_5_Data_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

4-1 구간 분할

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/파이썬 머신러닝 판다스 데이터 분석/data/part5/auto-mpg.csv')

In [5]:
df.columns = ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model year', 'origin', 'name']

print(df.head())

# horsepower의 누락 데이터를 삭제하고 실수형으로 변환
df['horsepower'].replace('?', np.nan, inplace = True)
df.dropna(subset = ['horsepower'], axis = 0, inplace = True)
df['horsepower'] = df['horsepower'].astype('float')

    mpg  cylinders  displacement horsepower  weight  acceleration  model year  \
0  15.0          8         350.0      165.0  3693.0          11.5          70   
1  18.0          8         318.0      150.0  3436.0          11.0          70   
2  16.0          8         304.0      150.0  3433.0          12.0          70   
3  17.0          8         302.0      140.0  3449.0          10.5          70   
4  15.0          8         429.0      198.0  4341.0          10.0          70   

   origin                name  
0       1   buick skylark 320  
1       1  plymouth satellite  
2       1       amc rebel sst  
3       1         ford torino  
4       1    ford galaxie 500  


In [6]:
# np.histogram 함수로 3개의 bin으로 구분할 경계값의 리스트 구하기
count, bin_dividers = np.histogram(df['horsepower'], bins = 3)
print(bin_dividers)

[ 46.         107.33333333 168.66666667 230.        ]


In [8]:
# 3개의 bin에 이름 저장
bin_names = ['저출력', ' 보통출력', '고출력']

# pd.cut 함수로 각 데이터를 3개의 bin에 할당
df['hp_bin'] = pd.cut(x = df['horsepower'],
                      bins = bin_dividers,
                      labels = bin_names,
                      include_lowest = True)
df[['horsepower', 'hp_bin']].head(15)

,horsepower,hp_bin
0,165.0,보통출력
1,150.0,보통출력
2,150.0,보통출력
3,140.0,보통출력
4,198.0,고출력
5,220.0,고출력
6,215.0,고출력
7,225.0,고출력
8,190.0,고출력
9,170.0,고출력


4-2 더미 변수: 범주형 데이터를 컴퓨터가 인식할 수 있도록 원핫인코딩

In [9]:
# hp_bin 열의 범주형 데이터를 더미 변수로 변환

horsepower_dummies = pd.get_dummies(df['hp_bin'])
horsepower_dummies.head()

,저출력,보통출력,고출력
0,0,1,0
1,0,1,0
2,0,1,0
3,0,1,0
4,0,0,1


In [20]:
# 원핫 인코딩
from sklearn import preprocessing

# 전처리를 위한 encoder 객체 만들기
label_encoder = preprocessing.LabelEncoder() # label_encoder 생성
onehot_encoder = preprocessing.OneHotEncoder() # one hot encoder 생성

# label_encoder로 문자열 범주를 숫자형 범주로 변환
onehot_labeled = label_encoder.fit_transform(df['hp_bin'].head(15))
print(onehot_labeled)
print(type(onehot_labeled))

# 2차월 행렬로 변경
onehot_reshaped = onehot_labeled.reshape(len(onehot_labeled),1)
print(onehot_reshaped)

# 희소행렬로 변환
onehot_fitted = onehot_encoder.fit_transform(onehot_reshaped)
print(onehot_fitted)
print(type(onehot_fitted))

[0 0 0 0 1 1 1 1 1 1 0 0 1 2 2]
<class 'numpy.ndarray'>
[[0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [2]
 [2]]
  (0, 0)	1.0
  (1, 0)	1.0
  (2, 0)	1.0
  (3, 0)	1.0
  (4, 1)	1.0
  (5, 1)	1.0
  (6, 1)	1.0
  (7, 1)	1.0
  (8, 1)	1.0
  (9, 1)	1.0
  (10, 0)	1.0
  (11, 0)	1.0
  (12, 1)	1.0
  (13, 2)	1.0
  (14, 2)	1.0
<class 'scipy.sparse.csr.csr_matrix'>


5 정규화

In [21]:
# horsepower 열틔 통계 요약 정보로 최댓값 확인
print(df['horsepower'].describe())

count    391.000000
mean     104.404092
std       38.518732
min       46.000000
25%       75.000000
50%       93.000000
75%      125.000000
max      230.000000
Name: horsepower, dtype: float64


Method 1. 최댓값의 절대값으로 나누기

In [22]:
# horsepower 열의 최대값의 절대값으로 모든 데이터를 나눠서 저장
df['horsepower'] = df['horsepower'] / abs(df.horsepower.max())

In [23]:
df['horsepower'].head()

0    0.717391
1    0.652174
2    0.652174
3    0.608696
4    0.860870
Name: horsepower, dtype: float64

In [24]:
df['horsepower'].describe()

count    391.000000
mean       0.453931
std        0.167473
min        0.200000
25%        0.326087
50%        0.404348
75%        0.543478
max        1.000000
Name: horsepower, dtype: float64

Method 2. (각 열의 데이터 값 - 최솟값) / (최댓값 - 최솟값)

In [25]:
min_x = df.horsepower - df.horsepower.min()
min_max = df.horsepower.max() - df.horsepower.min()
df.horsepower = min_x / min_max

print(df.horsepower.head())
print(df.horsepower.describe())

0    0.646739
1    0.565217
2    0.565217
3    0.510870
4    0.826087
Name: horsepower, dtype: float64
count    391.000000
mean       0.317414
std        0.209341
min        0.000000
25%        0.157609
50%        0.255435
75%        0.429348
max        1.000000
Name: horsepower, dtype: float64


6 시계열 데이터

6-1 다른 자료형을 시계열 객체로 변환

In [26]:
# to_datetime() ; 문자열 등 다른 자료형을 판다스 Timestamp를 나타내는 datetime64자료형으로 변환 가능

import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/파이썬 머신러닝 판다스 데이터 분석/data/part5/stock-data.csv')
df.head()

,Date,Close,Start,High,Low,Volume
0,2018-07-02,10100,10850,10900,10000,137977
1,2018-06-29,10700,10550,10900,9990,170253
2,2018-06-28,10400,10900,10950,10150,155769
3,2018-06-27,10900,10800,11050,10500,133548
4,2018-06-26,10800,10900,11000,10700,63039


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    20 non-null     object
 1   Close   20 non-null     int64 
 2   Start   20 non-null     int64 
 3   High    20 non-null     int64 
 4   Low     20 non-null     int64 
 5   Volume  20 non-null     int64 
dtypes: int64(5), object(1)
memory usage: 1.1+ KB


Method 1. 문자열 -> Timestamp

In [29]:
df['new_Date'] = pd.to_datetime(df['Date'])
df['new_Date'].head()

0   2018-07-02
1   2018-06-29
2   2018-06-28
3   2018-06-27
4   2018-06-26
Name: new_Date, dtype: datetime64[ns]

In [31]:
# 시계열 값으로 변환된 열을 새로운 행 인덱스로 지정
df.set_index('new_Date', inplace = True)
df.drop('Date', axis = 1, inplace = True) # 기존 열 삭제
df.head()

,Close,Start,High,Low,Volume
new_Date,,,,,
2018-07-02,10100,10850,10900,10000,137977
2018-06-29,10700,10550,10900,9990,170253
2018-06-28,10400,10900,10950,10150,155769
2018-06-27,10900,10800,11050,10500,133548
2018-06-26,10800,10900,11000,10700,63039


Method 2. Timestamp -> Period

In [32]:
# 날짜 형식의 문자열로 구성되는 리스트 정의
dates = ['2019-01-01', '2020-03-01', '2021-06-01']

# 문자열의 배열(시리즈 객체)을 판다스 Timestamp로 변환
ts_dates = pd.to_datetime(dates)
print(ts_dates)
print(type(ts_dates))

DatetimeIndex(['2019-01-01', '2020-03-01', '2021-06-01'], dtype='datetime64[ns]', freq=None)
<class 'pandas.core.indexes.datetimes.DatetimeIndex'>


In [37]:
# Timestamp를 Period로 변환
pr_day = ts_dates.to_period(freq = 'D') # 1일의 기간
print(pr_day)

pr_month = ts_dates.to_period(freq = 'M') # 1달의 기간
print(pr_month)

pr_year = ts_dates.to_period(freq = "A") # 1년의 기간
print(pr_year)

PeriodIndex(['2019-01-01', '2020-03-01', '2021-06-01'], dtype='period[D]')
PeriodIndex(['2019-01', '2020-03', '2021-06'], dtype='period[M]')
PeriodIndex(['2019', '2020', '2021'], dtype='period[A-DEC]')


6-2 시계열 데이터 만들기

Method 1. Timestamp 배열

In [38]:
import pandas as pd

# Timestamp의 배열 만들기 - 월 간격, 월의 시작일 기준
ts_ms = pd.date_range(start = '2019-01-01',
                      end = None,
                      periods = 6,
                      freq = "MS",
                      tz = "Asia/Seoul")
print(ts_ms)

DatetimeIndex(['2019-01-01 00:00:00+09:00', '2019-02-01 00:00:00+09:00',
               '2019-03-01 00:00:00+09:00', '2019-04-01 00:00:00+09:00',
               '2019-05-01 00:00:00+09:00', '2019-06-01 00:00:00+09:00'],
              dtype='datetime64[ns, Asia/Seoul]', freq='MS')


In [39]:
# 월 간격, 월의 마지막날 기준
ts_me = pd.date_range(start = '2019-01-01',
                      end = None,
                      periods = 6,
                      freq = "M",
                      tz = "Asia/Seoul")
print(ts_me)

DatetimeIndex(['2019-01-31 00:00:00+09:00', '2019-02-28 00:00:00+09:00',
               '2019-03-31 00:00:00+09:00', '2019-04-30 00:00:00+09:00',
               '2019-05-31 00:00:00+09:00', '2019-06-30 00:00:00+09:00'],
              dtype='datetime64[ns, Asia/Seoul]', freq='M')


In [42]:
# 분기(3개월) 간격, 월의 마지막날 기준
ts_3m = pd.date_range(start = '2019-01-01',
                      end = None,
                      periods = 6,
                      freq = "3M",
                      tz = "Asia/Seoul")
print(ts_3m)

DatetimeIndex(['2019-01-31 00:00:00+09:00', '2019-04-30 00:00:00+09:00',
               '2019-07-31 00:00:00+09:00', '2019-10-31 00:00:00+09:00',
               '2020-01-31 00:00:00+09:00', '2020-04-30 00:00:00+09:00'],
              dtype='datetime64[ns, Asia/Seoul]', freq='3M')


Method 2. Period 배열

In [43]:
import pandas as pd

# Period 배열 만들기 - 1개월 길이
pr_m = pd.period_range(start = '2019-01-01',
                       end = None,
                       periods = 3,
                       freq = 'M')
print(pr_m)

PeriodIndex(['2019-01', '2019-02', '2019-03'], dtype='period[M]')


In [44]:
# 2시간 길이
pr_h = pd.period_range(start = '2019-01-01',
                       end = None,
                       periods = 3,
                       freq = '2H')
print(pr_h)

PeriodIndex(['2019-01-01 00:00', '2019-01-01 02:00', '2019-01-01 04:00'], dtype='period[2H]')


6-3 시계열 데이터 활용

날짜 데이터 분리

In [71]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/파이썬 머신러닝 판다스 데이터 분석/data/part5/stock-data.csv')
df['new_Date'] = pd.to_datetime(df['Date'])
print(df.head())

         Date  Close  Start   High    Low  Volume   new_Date
0  2018-07-02  10100  10850  10900  10000  137977 2018-07-02
1  2018-06-29  10700  10550  10900   9990  170253 2018-06-29
2  2018-06-28  10400  10900  10950  10150  155769 2018-06-28
3  2018-06-27  10900  10800  11050  10500  133548 2018-06-27
4  2018-06-26  10800  10900  11000  10700   63039 2018-06-26


In [72]:
# dt 속성을 이용하여 new_Date 열의 연-월-일 정보를 년,월,일 로 구분
df['Year'] = df['new_Date'].dt.year
df['Month'] = df['new_Date'].dt.month
df['Day'] = df['new_Date'].dt.day
print(df.head())

         Date  Close  Start   High    Low  Volume   new_Date  Year  Month  Day
0  2018-07-02  10100  10850  10900  10000  137977 2018-07-02  2018      7    2
1  2018-06-29  10700  10550  10900   9990  170253 2018-06-29  2018      6   29
2  2018-06-28  10400  10900  10950  10150  155769 2018-06-28  2018      6   28
3  2018-06-27  10900  10800  11050  10500  133548 2018-06-27  2018      6   27
4  2018-06-26  10800  10900  11000  10700   63039 2018-06-26  2018      6   26


In [73]:
# to_period() 메소드를 적용하여 연-월-일 중에서 연-월 또는 연도를 추출
df['Date_yr'] = df['new_Date'].dt.to_period(freq = 'A')
df['Date_m'] = df['new_Date'].dt.to_period(freq = 'M')
print(df.head())

         Date  Close  Start   High    Low  Volume   new_Date  Year  Month  \
0  2018-07-02  10100  10850  10900  10000  137977 2018-07-02  2018      7   
1  2018-06-29  10700  10550  10900   9990  170253 2018-06-29  2018      6   
2  2018-06-28  10400  10900  10950  10150  155769 2018-06-28  2018      6   
3  2018-06-27  10900  10800  11050  10500  133548 2018-06-27  2018      6   
4  2018-06-26  10800  10900  11000  10700   63039 2018-06-26  2018      6   

   Day Date_yr   Date_m  
0    2    2018  2018-07  
1   29    2018  2018-06  
2   28    2018  2018-06  
3   27    2018  2018-06  
4   26    2018  2018-06  


날짜 인덱스 활용

In [74]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/파이썬 머신러닝 판다스 데이터 분석/data/part5/stock-data.csv')
df['new_Date'] = pd.to_datetime(df['Date'])
df.set_index('new_Date', inplace = True)

print(df.head())

                  Date  Close  Start   High    Low  Volume
new_Date                                                  
2018-07-02  2018-07-02  10100  10850  10900  10000  137977
2018-06-29  2018-06-29  10700  10550  10900   9990  170253
2018-06-28  2018-06-28  10400  10900  10950  10150  155769
2018-06-27  2018-06-27  10900  10800  11050  10500  133548
2018-06-26  2018-06-26  10800  10900  11000  10700   63039


In [75]:
df_y = df['2018']
df_y

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  """Entry point for launching an IPython kernel.


,Date,Close,Start,High,Low,Volume
new_Date,,,,,,
2018-07-02,2018-07-02,10100,10850,10900,10000,137977
2018-06-29,2018-06-29,10700,10550,10900,9990,170253
2018-06-28,2018-06-28,10400,10900,10950,10150,155769
2018-06-27,2018-06-27,10900,10800,11050,10500,133548
2018-06-26,2018-06-26,10800,10900,11000,10700,63039
2018-06-25,2018-06-25,11150,11400,11450,11000,55519
2018-06-22,2018-06-22,11300,11250,11450,10750,134805
2018-06-21,2018-06-21,11200,11350,11750,11200,133002
2018-06-20,2018-06-20,11550,11200,11600,10900,308596


In [76]:
df_ym = df.loc['2018-07']
df_ym

,Date,Close,Start,High,Low,Volume
new_Date,,,,,,
2018-07-02,2018-07-02,10100,10850,10900,10000,137977


In [77]:
df_ym_cols = df.loc['2018-07', 'Start':"High"]
df_ym_cols  

,Start,High
new_Date,,
2018-07-02,10850,10900


In [78]:
df_ymd = df['2018-07-02']
df_ymd

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  """Entry point for launching an IPython kernel.


,Date,Close,Start,High,Low,Volume
new_Date,,,,,,
2018-07-02,2018-07-02,10100,10850,10900,10000,137977


In [79]:
df_ymd_range = df['2018-06-25':'2018-06-20']
df_ymd_range

,Date,Close,Start,High,Low,Volume
new_Date,,,,,,


In [80]:
# 시간 간격 계산

today = pd.to_datetime(2018-12-25)
df['time_delta'] = today - df.index
df.head()

,Date,Close,Start,High,Low,Volume,time_delta
new_Date,,,,,,,
2018-07-02,2018-07-02,10100,10850,10900,10000,137977,-17714 days +00:00:00.000001981
2018-06-29,2018-06-29,10700,10550,10900,9990,170253,-17711 days +00:00:00.000001981
2018-06-28,2018-06-28,10400,10900,10950,10150,155769,-17710 days +00:00:00.000001981
2018-06-27,2018-06-27,10900,10800,11050,10500,133548,-17709 days +00:00:00.000001981
2018-06-26,2018-06-26,10800,10900,11000,10700,63039,-17708 days +00:00:00.000001981


In [81]:
df.set_index('time_delta', inplace = True)
df.head()
df_180 = df['180 days':'189 days']
df_180.head()

,Date,Close,Start,High,Low,Volume
time_delta,,,,,,
